In [1]:
from nj_crashes.paths import PUBLIC_DIR, NJDOT_DIR
from nj_crashes.crashes import Crashes
from utz import *

import duckdb

In [2]:
# orig_path = f'{PUBLIC_DIR}/njdot/with_cns.parquet'
orig_path = f'{NJDOT_DIR}/data/2021/NewJersey2021Accidents.pqt'

pqt_path = 'c21.parquet'
df = read_parquet(orig_path)
df.index.name = 'id'
# df['id'] = df.index.to_series()
df.to_parquet(pqt_path, index=True)
df

,County Code,Municipality Code,Department Case Number,County Name,Municipality Name,Police Department Code,Police Department,Police Station,Total Killed,Total Injured,Pedestrians Killed,Pedestrians Injured,Severity,Intersection,Alcohol Involved,HazMat Involved,Crash Type Code,Total Vehicles Involved,Crash Location,Location Direction,Route,Route Suffix,SRI (Standard Route Identifier),Mile Post,Road System,Road Character,Road Horizontal Alignment,Road Grade,Road Surface Type,Surface Condition,Light Condition,Environmental Condition,Road Divided By,Temporary Traffic Control Zone,Distance To Cross Street,Unit Of Measurement,Direction From Cross Street,Cross Street Name,Is Ramp,Ramp To/From Route Name,Ramp To/From Route Direction,Posted Speed,Posted Speed Cross Street,First Harmful Event,Latitude,Longitude,Cell Phone In Use Flag,Other Property Damage,Reporting Badge No.,Date
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,01,01,I-2020-005843,ATLANTIC,ABSECON CITY,01,ABSECON CITY PD,,0,1,0,0,I,B,False,False,11,1,US 30,E,30,,00000030__,53.89,02,,01,04,02,01,03,01,01,01,1056,FE,W,CR 646 / DELILAH RD,,,,50,25,54,NaN,NaN,N,CROSS PUSH BUTTON POLE TWO METAL ELECTRICAL C...,858,2021-03-21 19:14:00
1,01,01,I-2021-000001,ATLANTIC,ABSECON CITY,01,ABSECON CITY PD,,0,0,0,0,P,B,False,False,11,1,US 30,E,30,,00000030__,50.80,02,,03,04,02,01,06,01,01,01,175,FE,,,T,HADDON AVE,NB,45,,56,NaN,NaN,N,,846,2021-01-01 00:00:00
2,01,01,I-2021-000042,ATLANTIC,ABSECON CITY,01,ABSECON CITY PD,ABSECON,0,3,0,0,I,B,False,False,11,1,ATLANTIC COUNTY 646 II,,646,2,010006462_,0.13,05,,02,06,02,02,06,02,05,01,500,FE,W,US 30 / WHITE HORSE PIKE,,,,35,50,46,NaN,NaN,N,,854,2021-01-02 02:14:00
3,01,01,I-2021-000315,ATLANTIC,ABSECON CITY,01,ABSECON CITY PD,,0,0,0,0,P,B,False,False,11,1,US 30,W,30,,00000030__,53.59,02,,01,04,02,01,01,05,01,01,2640,FE,W,CR 646 / DELILAH RD,,,,50,25,48,NaN,NaN,N,,848,2021-01-05 14:08:00
4,01,01,I-2021-000527,ATLANTIC,ABSECON CITY,01,ABSECON CITY PD,,0,0,0,0,P,I,False,False,03,2,US 9,,9,,00000009__,42.74,02,,01,04,02,01,01,01,05,01,,AT,,CR 630 / OHIO AVE,,,,35,35,26,NaN,NaN,N,,866,2021-01-07 15:35:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226953,21,23,B150-2021-00117A,WARREN,WHITE TWP,02,NEW JERSEY STATE POLICE,WASHINGTON - SQ,0,0,0,0,P,I,False,False,11,1,WARREN COUNTY 620,N,620,,21000620__,0.00,05,,02,04,02,01,06,01,05,01,,AT,,ROUTE 519 / S BRIDGEVILLE RD,,,,50,50,46,40.80707,75.07559,N,Damage to 10ft of Country road guardrail.,7837,2021-05-26 21:08:00
226954,21,23,B150-2021-00118A,WARREN,WHITE TWP,02,NEW JERSEY STATE POLICE,WASHINGTON - SQ,0,0,0,0,P,I,False,False,01,2,MANUNKA CHUNK RD / UPPER SEREPTA RD **,W,,,21231034__,1.73,07,,01,04,02,01,01,01,05,01,,AT,,MASSENATTS RD,,,,40,50,26,40.84466,75.05549,N,,8610,2021-05-25 07:26:00
226955,21,23,B150-2021-00128A,WARREN,WHITE TWP,02,NEW JERSEY STATE POLICE,WASHINGTON - SQ,0,0,0,0,P,B,False,False,12,1,ROUTE 519,S,519,,00000519__,44.00,05,,03,04,02,02,05,04,05,01,,,,,,,,50,,24,40.80564,75.07627,N,,8514,2021-06-03 20:17:00


In [3]:
# db_path = 'crashes.duckdb'
db_path = '2021.duckdb'
con = duckdb.connect(db_path)
dsql = con.sql

In [4]:
def tables():
    return dsql("SHOW TABLES").df()

In [5]:
def indices():
    return dsql("SELECT * from duckdb_indexes()").df()

In [6]:
tbls = tables()
if not tbls[tbls.name == 'crashes'].empty:
    dsql("DROP TABLE crashes")
tables()

,name


In [7]:
indices()

,database_name,database_oid,schema_name,schema_oid,index_name,index_oid,table_name,table_oid,is_unique,is_primary,expressions,sql


In [8]:
dsql("VACUUM ANALYZE")

In [13]:
def db_size():
    return dsql("PRAGMA database_size").df()

In [11]:
con.close()

In [12]:
%%time
dsql(f"CREATE TABLE crashes AS SELECT * FROM '{pqt_path}';")

CPU times: user 713 ms, sys: 37.9 ms, total: 750 ms
Wall time: 787 ms


In [14]:
db_size()

,database_name,database_size,block_size,total_blocks,used_blocks,free_blocks,wal_size,memory_usage,memory_limit
0,2021,14.6MB,262144,56,54,2,0 bytes,14.1MB,13.7GB


In [15]:
%%time
dsql(f"CREATE UNIQUE INDEX crashes_id_idx ON crashes (id);")

CPU times: user 56.6 ms, sys: 14.4 ms, total: 71 ms
Wall time: 64.5 ms


In [16]:
dsql("CHECKPOINT")

┌─────────┐
│ Success │
│ boolean │
├─────────┤
│ 0 rows  │
└─────────┘

In [12]:
%%time
dsql(f"CREATE INDEX crashes_dt_icn_severity_idx ON crashes (Date, \"County Code\", severity);")

CPU times: user 201 ms, sys: 11.6 ms, total: 213 ms
Wall time: 87.7 ms


In [14]:
con.commit()

In [17]:
con.close()

In [28]:
# %%time
# dsql("EXPORT DATABASE 'crashes.duckdb'")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: user 14.1 s, sys: 4.19 s, total: 18.3 s
Wall time: 5.62 s


In [36]:
dsql("SELECT * FROM crashes WHERE id=100000")

┌───────┬───────┬───────┬─────────┬─────────┬───┬─────────┬─────────┬────────┬────────┬─────────┬─────────┐
│ year  │  cc   │  mc   │  case   │   cn    │ … │   ocn   │ reason  │  ilon  │  ilat  │   icn   │   id    │
│ int64 │ int64 │ int64 │ varchar │ varchar │   │ varchar │ varchar │ double │ double │ varchar │  int64  │
├───────┼───────┼───────┼─────────┼─────────┼───┼─────────┼─────────┼────────┼────────┼─────────┼─────────┤
│  2004 │     2 │    63 │ 04-077  │ Bergen  │ … │ NULL    │ No MP   │   NULL │   NULL │ NULL    │ 1000000 │
├───────┴───────┴───────┴─────────┴─────────┴───┴─────────┴─────────┴────────┴────────┴─────────┴─────────┤
│ 1 rows                                                                            57 columns (11 shown) │
└─────────────────────────────────────────────────────────────────────────────────────────────────────────┘

In [37]:
dsql("DESCRIBE crashes")

┌───────────────────────────┬─────────────┬─────────┬─────────┬─────────┬───────┐
│        column_name        │ column_type │  null   │   key   │ default │ extra │
│          varchar          │   varchar   │ varchar │ varchar │ varchar │ int32 │
├───────────────────────────┼─────────────┼─────────┼─────────┼─────────┼───────┤
│ year                      │ BIGINT      │ YES     │ NULL    │ NULL    │  NULL │
│ cc                        │ BIGINT      │ YES     │ NULL    │ NULL    │  NULL │
│ mc                        │ BIGINT      │ YES     │ NULL    │ NULL    │  NULL │
│ case                      │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ cn                        │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ mn                        │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ pdc                       │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ pdn                       │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ station       

In [40]:
%%time
dsql("""
SELECT date_part('year', dt) as year, icn as county, severity, count(*) as num
FROM crashes
WHERE icn IS NOT NULL
GROUP BY (date_part('year', dt), icn, severity)
ORDER BY (date_part('year', dt), icn, severity)
""")

CPU times: user 771 µs, sys: 26 µs, total: 797 µs
Wall time: 809 µs


┌───────┬────────────┬──────────┬───────┐
│ year  │   county   │ severity │  num  │
│ int64 │  varchar   │ varchar  │ int64 │
├───────┼────────────┼──────────┼───────┤
│  2001 │ Atlantic   │ f        │    49 │
│  2001 │ Atlantic   │ i        │  1689 │
│  2001 │ Atlantic   │ p        │  2860 │
│  2001 │ Bergen     │ f        │    16 │
│  2001 │ Bergen     │ i        │  3181 │
│  2001 │ Bergen     │ p        │  7189 │
│  2001 │ Burlington │ f        │    54 │
│  2001 │ Burlington │ i        │  1829 │
│  2001 │ Burlington │ p        │  4050 │
│  2001 │ Camden     │ f        │    33 │
│    ·  │   ·        │ ·        │     · │
│    ·  │   ·        │ ·        │     · │
│    ·  │   ·        │ ·        │     · │
│  2021 │ Somerset   │ p        │  5506 │
│  2021 │ Sussex     │ f        │     9 │
│  2021 │ Sussex     │ i        │   423 │
│  2021 │ Sussex     │ p        │  1591 │
│  2021 │ Union      │ f        │    28 │
│  2021 │ Union      │ i        │  3698 │
│  2021 │ Union      │ p        │ 

In [38]:
dsql("EXPLAIN SELECT year, count(*) as num FROM crashes GROUP BY year")

┌───────────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────┐
│  explain_key  │                                            explain_value                                             │
│    varchar    │                                               varchar                                                │
├───────────────┼──────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ physical_plan │ ┌───────────────────────────┐\n│         PROJECTION        │\n│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │\n│__in…  │
└───────────────┴──────────────────────────────────────────────────────────────────────────────────────────────────────┘

In [39]:
dsql("EXPLAIN ANALYZE SELECT * FROM crashes WHERE id=1000000")

┌───────────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────┐
│  explain_key  │                                            explain_value                                             │
│    varchar    │                                               varchar                                                │
├───────────────┼──────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ analyzed_plan │ ┌─────────────────────────────────────┐\n│┌───────────────────────────────────┐│\n││    Query Prof…  │
└───────────────┴──────────────────────────────────────────────────────────────────────────────────────────────────────┘